In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pickle.load(open('dataset_level3.pkl', 'rb'))

In [3]:
data.head()

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five,runs_x
12662,Sri Lanka,Pakistan,Lahore,160,10,4,8.73,44.0,182
48668,England,Sri Lanka,Delhi,116,23,7,7.18,42.0,171
38526,West Indies,New Zealand,Wellington,66,60,7,6.60,35.0,159
3089,Pakistan,West Indies,Trinidad,119,21,5,7.21,29.0,137
18192,New Zealand,England,Manchester,54,62,7,5.59,36.0,123


In [4]:
X = data.drop(columns=['runs_x'], axis=1)
y = data['runs_x']

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [6]:
X.head()

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
12662,Sri Lanka,Pakistan,Lahore,160,10,4,8.73,44.0
48668,England,Sri Lanka,Delhi,116,23,7,7.18,42.0
38526,West Indies,New Zealand,Wellington,66,60,7,6.60,35.0
3089,Pakistan,West Indies,Trinidad,119,21,5,7.21,29.0
18192,New Zealand,England,Manchester,54,62,7,5.59,36.0


In [7]:
y.value_counts()

153    1151
147     858
168     764
150     761
191     759
       ... 
112      64
107      61
97       40
77       26
80       15
Name: runs_x, Length: 134, dtype: int64

In [8]:
X_train

,batting_team,bowling_team,city,current_score,balls_left,wickets_left,crr,last_five
42764,New Zealand,Pakistan,Dubai,54,70,8,6.48,38.0
38374,New Zealand,West Indies,Auckland,45,90,9,9.00,45.0
26156,Pakistan,Australia,St Lucia,106,41,8,8.05,34.0
11614,India,West Indies,Lauderhill,34,93,10,7.56,30.0
27707,Australia,England,Melbourne,62,81,9,9.54,57.0
...,...,...,...,...,...,...,...,...
26655,New Zealand,Australia,Christchurch,165,18,4,9.71,69.0
10383,India,Bangladesh,Nagpur,139,23,6,8.60,50.0
7794,Bangladesh,West Indies,Lauderhill,131,33,6,9.03,34.0
37029,Sri Lanka,Bangladesh,Pallekele,123,29,6,8.11,34.0


### Make a Pipeline

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_absolute_error

In [10]:
trf = ColumnTransformer([
    ('trf', OneHotEncoder(sparse=False, drop='first'), ['batting_team', 'bowling_team', 'city'])
]
, remainder='passthrough')

In [11]:
pipe = Pipeline(steps=[
    ('step1', trf),
    ('step2', StandardScaler()),
    ('step3', XGBRegressor(n_estimators=1000, learning_rate=0.2, max_depth=12, random_state=1))
])

In [12]:
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(r2_score(y_test, y_pred))
print(mean_absolute_error(y_test, y_pred))

0.9880234289939701
1.658021018807457


In [14]:
pickle.dump(pipe, open('model/pipe.pkl', 'wb'))